In [34]:
import ipywidgets as widgets
import IPython.display as display
import threading as Thread
from queue import Queue 

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description = "Record",
    disabled=False,
    button_style = "Success",
    icon="microphone"
)

stop_button = widgets.Button(
    description = "Stop",
    disabled=False,
    button_style = "Warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
     
    with output:
        display.display('Recording')
        record = Thread.Thread(target=record_microphone)
        record.start()
            
        transcribe = Thread.Thread(target=speech_recog,args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display.display("Stopped")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display.display(record_button,stop_button,output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [35]:
import pyaudio
p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (3- USB PnP Audio De', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microphone (NVIDIA Broadcast)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannel

In [36]:
CHANNELS = 1
FRAME_RATE = 48000
RECORD_SECONDS = 20 
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(channels=CHANNELS,
                   rate=FRAME_RATE,
                   format=AUDIO_FORMAT,
                   input=True,
                   input_device_index=15,
                   frames_per_buffer=chunk)
    
    frames=[]
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= (FRAME_RATE*RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames=[]
                

    stream.stop_stream()
    stream.close()
    p.terminate()


In [37]:
import json
import subprocess
from vosk import Model, KaldiRecognizer

model = Model(model_name='vosk-model-en-us-0.22')
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recog(output):
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text=json.loads(result)['text']
        
        cased = subprocess.check_output("python recasepunc/recasepunc.py predict recasepunc/checkpoint", shell=True,text=True,input=text)
        output.append_stdout(cased)    
            
        
    